## Including Markdown

#### Attempt #1 - unsuccessful

In [1]:
import markdown
from markdown_include.include import MarkdownInclude
from pathlib import Path

In [2]:
!pip list | grep markdown

markdown-include      0.7.0
You should consider upgrading via the '/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/.venv_dev_michael-booth-phd-thesis/bin/python3 -m pip install --upgrade pip' command.


The syntax for use within your Markdown files is `{!filename!}`. This statement will be replaced by the contents of `filename`.

The following settings can be specified when initialising the plugin:
* `base_path`: Default location from which to evaluate relative paths for the include statement. (Default: the run-directory.)
* `encoding`: Encoding of the files used by the include statement. (Default: utf-8.)
* `inheritHeadingDepth` : If true, increases headings on include file by amount of previous heading. Combines with headingOffset option, below. (Default: False.)
* `headingOffset`: Increases heading depth by a specific amount, in addition to the inheritHeadingDepth Option. (Default: 0)
* `throwException`: When true, if the extension is unable to find an included file it will throw an exception which the user can catch. If false (default), a warning will be printed and Markdown will continue parsing the file.

In [3]:
def process_md_file(source_md_file, base_path=".",  encoding="iso-8859-1"):
    with open(source_md_file, "r") as md:
        source_md = md.readlines()
    markdown_include = MarkdownInclude(
        configs={'base_path': base_path, 'encoding': encoding}
    )
    html = markdown.markdown(str(source_md), extensions=[markdown_include])
    html_file = source_md_file.replace(".md", ".html")
    html = html.replace("[", "").replace("]", "")
    with open(html_file, "w") as ht:
        ht.write(html)
    return html

In [4]:
test_md_filename = Path.cwd().parent / "app_theory.md"

In [5]:
test_html_filename = test_md_filename.as_posix().replace(".md", ".html")

In [6]:
test_html_filename

'/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/docs/app_theory.html'

In [7]:
test_md_filename.as_posix()

'/Users/mjboothaus/code/github/mjboothaus/PhD-Thesis/docs/app_theory.md'

In [8]:
test_html = process_md_file(test_md_filename.as_posix())

In [9]:
test_html

'<p>\'### Theory for the interface\\n\', \'\\n\', \'To predict the normalized probability $g_{wi}(z)$ for finding an oxygen ($i$ = O) or\\n\', \'hydrogen ($i$ = H) atom of a water molecule at a distance $z$ perpendicular to the\\n\', \'planar interface ("wall"), we use the "singlet" theory version of the Ornstein-Zernike\\n\', \'equation due to Henderson, Abraham and Barker and Percus\\n\', \'together with the approximate hypernetted chain (HNC) closure. We have also used this level of theory to study soft-core primitive model electrolytes and molten salts next to charged interfaces.\\n\', \'\\n\', \'The input to the calculation is:\\n\', \'1. the structure of the bulk model water, as contained in the direct correlation functions $c_{ij}(r)$, for all distinct pairs of species, $ij$ = OO, OH and HH, separated by a distance $r$\\n\', \'2. the bulk water density $\\bar{\\rho}$ and temperature $T$\\n\', \'3. the short-range potential energy between the wall and the oxygen and hydrogen atom

In [10]:
from tidylib import tidy_document

In [11]:
document, errors = tidy_document(test_html, options={'numeric-entities':1})

In [12]:
errors

"line 1 column 1 - Warning: missing <!DOCTYPE> declaration\nline 1 column 1 - Warning: inserting missing 'title' element\n"

In [13]:
document

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN">\n<html>\n  <head>\n    <title></title>\n  </head>\n  <body>\n    <p>\n      \'### Theory for the interface\\n\', \'\\n\', \'To predict the normalized probability $g_{wi}(z)$ for finding an oxygen ($i$ = O) or\\n\', \'hydrogen ($i$ = H) atom of a water molecule at a distance $z$ perpendicular to the\\n\', \'planar interface ("wall"), we use the "singlet" theory version of the Ornstein-Zernike\\n\', \'equation due to Henderson, Abraham and Barker and Percus\\n\', \'together with the approximate hypernetted chain (HNC) closure. We have also used this level of theory to study soft-core primitive model electrolytes and molten salts next to charged interfaces.\\n\', \'\\n\', \'The input to the calculation is:\\n\', \'1. the structure of the bulk model water, as contained in the direct correlation functions $c_{ij}(r)$, for all distinct pairs of species, $ij$ = OO, OH and HH, separated by a distance $r$\\n\', \'2. the bulk water density $\\b

#### Attempt #2

In [14]:
import regex as re

In [15]:
def include_md(md_filename, working_dir):
    with open(md_filename, "r") as file:
        filedata = file.read()
    re_filename_md = r'(?<={!)(.*)(?=!})'

    match_file = re.findall(re_filename_md, filedata, flags=re.MULTILINE)
    for file_md in match_file:
        with open(working_dir / file_md, "r") as inc_file:
            include_md = inc_file.read()
            include_md = include_md.replace("\\", "BACKSLASH")
            filedata = re.sub("{!" + file_md + "!}", re.escape(include_md, special_only=True), filedata, flags=re.MULTILINE)

    md_filename_inc = md_filename.as_posix().replace(".md", "_inc.md")

    with open(md_filename_inc, "w") as file:
        file.write(filedata)

    with open(working_dir / md_filename_inc, "r") as inc_file:
        md = inc_file.read()
        md = md.replace("\\", "").replace("BACKSLASH", "\\")


    with open(working_dir / md_filename_inc, "w") as inc_file:
        inc_file.write(md)

In [16]:
working_dir = Path.cwd().parent

In [17]:
include_md(test_md_filename, working_dir)